In [ ]:
import pandas as pd

import pandasql as pdsql

import sqlalchemy
import psycopg2

import data

In [ ]:
df_runup = pd.read_excel('runup_only_postgres.xlsx')

In [ ]:
df_runup.head(5)

In [ ]:
user_name = 'user'
password = 'p'
server = df_runup['SERVER_NAME']
dns = df_runup['DNS']
db =  df_runup['IR_NAME']
port = '5433' # PGSE 5433 дефолтный
stand_type = df_runup['STAND_TYPE']
CI_DB = df_runup['IR_ID']

list_cosyachnih_str = []

# Это простейший запрос к БД к таблице с метаданными
query_all = "SELECT r.rolname as users, ARRAY(SELECT b.rolname FROM pg_catalog.pg_auth_members m JOIN pg_catalog.pg_roles b ON (m.roleid = b.oid) WHERE m.member = r.oid) as roles FROM pg_catalog.pg_roles r where 1=1 AND r.rolname !~ '^pg_' ORDER BY 1;"

# Формируем наименование текстового файла с косячными строками подключения с пометкой "STAND_TYPE" из переменной sheet_name_fact
#name_kosyach_txt = sheet_name_fact + '_косячные_строки_подключ.txt'
#name_kosyach_xlsx = sheet_name_fact + '_косячные_строки_подключ.xlsx'

# Создаем пустой df,куда будем писать косячные строки подключения
df_all_final = pd.DataFrame(columns=['STAND_TYPE', 'URL', 'CI_DB', 'db_name', 'as_admin', 'as_admin_read', 'as_TUZ', 'other'])

In [ ]:
for i in range(len(server)):
    con_string = 'postgresql+psycopg2://' + str(user_name) + ":" + str(password) + "@" + str(server[i]) + '.' + str(dns[i]) + ":" + str(port) + "/" + str(db[i]) 
    try:
        con = sqlalchemy.create_engine(str(con_string))
        df_iter = pd.DataFrame(con.execute(query_all).fetchall()) # column = col  
        df_iter =  df_iter.rename(columns={df_iter.columns[0]: 'users', df_iter.columns[1]: 'roles'})
        df_iter = df_iter.astype(str)

        df_as_TUZ = df_iter[df_iter['roles'].str.contains('as_TUZ', regex=True, na=True)] 
        #df_as_admin = df_iter[df_iter['roles'].str.contains('as_admin', regex=True, na=True)]
        df_as_admin = pdsql.sqldf("select distinct * from df_iter where roles like \"%'as_admin'%\";", locals())
        df_as_admin_read = df_iter[df_iter['roles'].str.contains('as_admin_read', regex=True, na=True)]
        df_other = pdsql.sqldf("select distinct * from df_iter where roles NOT like \"%as_TUZ%\" AND roles NOT like \"%as_admin%\";", locals())


        if len(df_as_TUZ) > 0:
            list_users_as_TUZ = df_as_TUZ['users'].to_list()   
            str_users_as_TUZ_0 = str(list_users_as_TUZ)
            str_users_as_TUZ_1 = str_users_as_TUZ_0.replace("'", "")
            str_users_as_TUZ_2 = str_users_as_TUZ_1.replace('[', '')
            str_users_as_TUZ_3 = str_users_as_TUZ_2.replace(']', '')
        else: str_users_as_TUZ_3 = None

        if len(df_as_admin) > 0:
            list_users_as_admin = df_as_admin['users'].to_list()   
            str_users_as_admin_0 = str(list_users_as_admin)
            str_users_as_admin_1 = str_users_as_admin_0.replace("'", "")
            str_users_as_admin_2 = str_users_as_admin_1.replace('[', '')
            str_users_as_admin_3 = str_users_as_admin_2.replace(']', '')
        else: str_users_as_admin_3 = None  

        if len(df_as_admin_read) > 0:
            list_users_as_admin_read = df_as_admin_read['users'].to_list()   
            str_users_as_admin_read_0 = str(list_users_as_admin_read)
            str_users_as_admin_read_1 = str_users_as_admin_read_0.replace("'", "")
            str_users_as_admin_read_2 = str_users_as_admin_read_1.replace('[', '')
            str_users_as_admin_read_3 = str_users_as_admin_read_2.replace(']', '')
        else: str_users_as_admin_read_3 = None        

        if len(df_other) > 0:
            list_users_other = df_other['users'].to_list()   
            str_users_other_0 = str(list_users_other)
            str_users_other_1 = str_users_other_0.replace("'", "")
            str_users_other_2 = str_users_other_1.replace('[', '')
            str_users_other_3 = str_users_other_2.replace(']', '')
        else: str_users_other_3 = None   

        df_all_final = df_all_final.append({'STAND_TYPE':stand_type[i], 'URL':str(server[i]) + '.'+ str(dns[i]),
                                            'CI_DB':CI_DB[i], 'db_name':db[i],'as_admin':str_users_as_admin_3,
                                            'as_admin_read':str_users_as_admin_read_3,'as_TUZ':str_users_as_TUZ_3,
                                            'other':str_users_other_3}, ignore_index=True)
    except:
        print(i, ' | ', con_string)   
        with open('Косячные_строки_подключения.txt', 'a') as f:
            f.write(con_string +' \n')
    
df_all_final.to_excel('ALL_users_new_pop1.xlsx')     
        

In [ ]:
df_as_admin = pdsql.sqldf("select distinct * from df_iter where roles like \"%'as_admin'%\";", locals())

In [ ]:
df_as_admin

In [ ]:
df_iter